CÉLULA DE MARKDOWN EXEMPLO

## Explicação ##

* anotar textos/ apresentar

favor rodar{ pip install -r requirements.txt }

In [ ]:
#Célula de código
!pip install geopy #comentar essa linha depois de rodar primeira vez, não me pergunte o pq
import pandas as pd
from geopy.distance import geodesic